In [416]:
import os
import re
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
stopwords = open("listfile.txt").read().splitlines()
def loadData(loc):
        BASEDIR=os.path.dirname(os.path.realpath('__file__'))
        df=pd.read_csv(BASEDIR+loc,names=["headline","news"])
        return df


In [417]:
df=loadData("/dataset/onlineKhabar.csv")
REPLACE_BY_SPACE_RE = re.compile('[?!/(){}\[\]\|@,;\'\']')

In [418]:
def removeCharacs(text):
    text=re.sub(REPLACE_BY_SPACE_RE,' ',text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    return text

In [419]:
flatten = lambda x : ''.join(x.split('\','))

In [420]:
df['news'] = df['news'].map(flatten).map(removeCharacs)


In [464]:
def stem_text_cat2(text):
    return re.sub('(को|ले|बाट|का|हरु|हरुसँग|सँग|लाई|हरू|हरूसँग|हरू|पटक|मध्ये|भन्दा|देखि)','',text)

In [422]:
allwords = open("dict.txt").read().splitlines()

In [423]:
suffixes = ('ी' , 'ीया' ,  'ीय' ,'े' , 'नु' , 'दै' , 'ेको' , 'ै','कै','मा')
def stem_text_cat1(word):
    if word.endswith(suffixes):
        wordlist =  [word.rstrip(x) for x in suffixes]
        for i in wordlist:
            if i!= word:
                if i in allwords:
                    return i
                    
        
    return word
    

In [424]:
def stemming(text):
    text =  ' '.join([stem_text_cat1(a) for a in word_tokenize(text)])
    return ' '.join([stem_text_cat2(a) for a in word_tokenize(text)])
    

In [425]:
df['news'] = df['news'].map(stemming)
df['fk'] = df['news']

In [426]:
def sent_tokenize(text):
    sentence = [x for x in text.split('।')] 
    return sentence

In [427]:
df['news']=df['news'].map(sent_tokenize)

In [428]:
df['original'] = df['news']

In [429]:
def check(x):
    try:
        if isinstance(float(x), float):
            return False
    except:
        return True


In [430]:
# function to remove stopwords
def remove_stopword(sen):
    sen_new = " ".join([i for i in sen if i not in stopwords and check(i)])
    return sen_new

def remove_stopword_wnum(sen):
    sen_new = " ".join([i for i in sen if i not in stopwords])
    return sen_new

In [431]:
remove_stopwords = lambda sent : [remove_stopword(r.split()) for r in sent]
remove_stopwords_wnum = lambda sent : [remove_stopword_wnum(r.split()) for r in sent]
df['news']=df['news'].map(remove_stopwords)
df['news-w-num'] = df['news'].map(remove_stopwords_wnum)

# Sentence Extraction Using PageRank

In [432]:
from sklearn.feature_extraction.text import TfidfVectorizer
def tfidf(corpus):
    vectorizer = TfidfVectorizer(max_df = 5 , min_df = 1)
    X = vectorizer.fit_transform(corpus)
    stopwords = vectorizer.get_stop_words()
    return X

In [433]:
df['news_vector']=df['news'].map(tfidf)
from sklearn.metrics.pairwise import cosine_similarity

In [434]:
def similarity(sentences):
    sim_mat = np.zeros([sentences.shape[0],sentences.shape[0]])
    for i in range(sentences.shape[0]):
          for j in range(sentences.shape[0]):
                if i != j:
                      sim_mat[i][j] = cosine_similarity(sentences[i], sentences[j])[0,0]
                        
    return sim_mat

In [435]:
df['similarity']=df['news_vector'].map(similarity)

In [436]:
import networkx as nx
ranks=[]
for i in range(len(df)):
    nx_graph = nx.from_numpy_array(df['similarity'][i])
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[k],s) for k,s in enumerate(df['original'][i])), reverse=True)
    ranks.append((ranked_sentences[0:3]))
    

In [437]:
summaries = pd.DataFrame(ranks,columns = ['Summary1','Summary2','Summary3'])

In [438]:
summaries['Summary1'][0]

(0.11761220664537599,
 ' गत वर्ष सन् २०१५ एप्पलल बिक्रि २८ प्रतिशत बढ्यो भन मुनाफा ३५ प्रतिशत बढे बताइए छ ')

# Keyword Extraction

In [439]:
df['news'][0]

['विश्व नम्वर वान टेक कम्पनी एप्पल मानिस प्रमुख र्यरी अधिकृत सिइओ पाउन तलब भत्ता चानचुने कुरै',
 'एप्पल वर्तमान सिइओ टिम कुक वर्ष सन् अर्ब रुपैयाँ बढि वाषिर्क तलबभत्ता',
 'अघिल्लो वर्ष तुलनामा सन् कुक तलब भत्तामा प्रतिशत वृद्धि',
 'वर्षमा एप्पल करोड डलर रकम',
 'कुक आधारभुत बेतनमा वर्ष प्रतिशत करोड रुपैयाँ वृद्धि नन इक्विटी इन्सेन्टिभमा प्रतिशत वृद्धि करोड रुपैयाँ',
 'यतिधेरै तलब भत्ता थाप्न कुक एप्पल एकमात्र कर्मचारी होलान् कर्मचारी तलब निक सोँच्नुहोला',
 'अचम्म एप्पल उच्च पदस्थ र्यरी कुक तलब भत्ता',
 'कम्पनी प्रमुख वित्तिय अधिकृत लू वाषिर्क तलबमा प्रतिशत वृद्धि वर्ष अढाइ अर्ब रुपैयाँ बढि तलब भत्ता',
 'रिटेल एण्ड अनलाइन स्टोर वरिष्ठ उपाध्यक्ष एन्जेला वर्ष एप्पलमा बढि तलब भत्ता बुझ्ने कर्मचारी',
 'वर्ष अर्ब करोड रुपैयाँ बढि तलब भत्ता',
 'वर्ष सन् एप्पलल बिक्रि प्रतिशत बढ्यो मुनाफा प्रतिशत बढे बताइए',
 'सन् एप्पलल शेयर मूल्यमा गिरावट',
 '']

In [440]:
# function to remove stopwords
def remove_stopword(sen):
    sen_new = " ".join([i for i in sen if i not in stopwords and check(i)])
    return sen_new
remove_stopwords = lambda sent : [remove_stopword(r.split()) for r in sent]

In [441]:
def remove_stopwords_w_num(sen):
    sen_new = " ".join([i for i in sen if i not in stopwords])
    return sen_new
remove_stopwords_w_num = lambda sent : [remove_stopword(r.split()) for r in sent]

In [442]:
df['news-ws'] = df['news'].map(remove_stopwords)
df['news-w-num'] = df['news-w-num'].map(remove_stopwords_w_num)

In [443]:
df['news-w-num'][0]

['विश्व नम्वर वान टेक कम्पनी एप्पल मानिस प्रमुख र्यरी अधिकृत सिइओ पाउन तलब भत्ता चानचुने कुरै',
 'एप्पल वर्तमान सिइओ टिम कुक वर्ष सन् अर्ब रुपैयाँ बढि वाषिर्क तलबभत्ता',
 'अघिल्लो वर्ष तुलनामा सन् कुक तलब भत्तामा प्रतिशत वृद्धि',
 'वर्षमा एप्पल करोड डलर रकम',
 'कुक आधारभुत बेतनमा वर्ष प्रतिशत करोड रुपैयाँ वृद्धि नन इक्विटी इन्सेन्टिभमा प्रतिशत वृद्धि करोड रुपैयाँ',
 'यतिधेरै तलब भत्ता थाप्न कुक एप्पल एकमात्र कर्मचारी होलान् कर्मचारी तलब निक सोँच्नुहोला',
 'अचम्म एप्पल उच्च पदस्थ र्यरी कुक तलब भत्ता',
 'कम्पनी प्रमुख वित्तिय अधिकृत लू वाषिर्क तलबमा प्रतिशत वृद्धि वर्ष अढाइ अर्ब रुपैयाँ बढि तलब भत्ता',
 'रिटेल एण्ड अनलाइन स्टोर वरिष्ठ उपाध्यक्ष एन्जेला वर्ष एप्पलमा बढि तलब भत्ता बुझ्ने कर्मचारी',
 'वर्ष अर्ब करोड रुपैयाँ बढि तलब भत्ता',
 'वर्ष सन् एप्पलल बिक्रि प्रतिशत बढ्यो मुनाफा प्रतिशत बढे बताइए',
 'सन् एप्पलल शेयर मूल्यमा गिरावट',
 '']

In [444]:
from nltk import word_tokenize

In [445]:
vocab = set([x for x in ' '.join(df['news-ws'][19]).split(' ') ])
vocab = list(vocab)

In [446]:
def checkMe(x):
    try:
        if isinstance(float(x), float):
            return True
    except:
        return False
def checkNum(x):
    try:
        if isinstance(float(x), float):
            return 
    except:
        return x

def findCandidates(text):
    clusters=[]
    word1=[]
    for word in word_tokenize(text):
        if (word in stopwords) or checkMe(word):
            clusters.append(word1)
            word1 =[]
            continue
        else:
            word1.append(word)
    return [' '.join(a).replace('।','') for a in clusters if len(a)>1]
df['kW-candidates'] = df['fk'].map(findCandidates)

In [447]:
df['kW-candidates'][0]

['विश्व नम्वर वान टेक कम्पनी एप्पल',
 'प्रमुख र्यरी अधिकृत सिइओ पाउन तलब भत्ता चानचुने',
 'एप्पल वर्तमान सिइओ टिम कुक',
 'वर्ष सन्',
 'अर्ब रुपैयाँ बढि वाषिर्क तलबभत्ता',
 ' अघिल्लो वर्ष तुलनामा सन्',
 'कुक तलब भत्तामा',
 'प्रतिशत वृद्धि',
 'वर्षमा एप्पल',
 'डलर रकम',
 ' कुक आधारभुत बेतनमा',
 'करोड रुपैयाँ वृद्धि',
 'नन इक्विटी इन्सेन्टिभमा',
 'प्रतिशत वृद्धि',
 'करोड रुपैयाँ',
 'यतिधेरै तलब भत्ता थाप्न कुक एप्पल एकमात्र कर्मचारी होलान्',
 'कर्मचारी तलब',
 'सोँच्नुहोला ',
 'एप्पल उच्च पदस्थ र्यरी कुक',
 'तलब भत्ता',
 ' कम्पनी प्रमुख वित्तिय अधिकृत लू वाषिर्क तलबमा',
 'प्रतिशत वृद्धि',
 'वर्ष अढाइ अर्ब रुपैयाँ बढि तलब भत्ता',
 ' रिटेल एण्ड अनलाइन स्टोर वरिष्ठ उपाध्यक्ष एन्जेला',
 'वर्ष एप्पलमा',
 'बढि तलब भत्ता बुझ्ने कर्मचारी',
 'करोड रुपैयाँ बढि तलब भत्ता',
 'वर्ष सन्',
 'एप्पलल बिक्रि',
 'प्रतिशत बढ्यो',
 'प्रतिशत बढे बताइए',
 'एप्पलल शेयर मूल्यमा']

In [448]:
from collections import OrderedDict

In [449]:
def find_scores(text):
    document = [checkNum((word_tokenize(x))) for x in text]
    vocab = set([x for x in ' '.join(text).split(' ') ])
    vocab = list(vocab)
    names = vocab
    occurrences = OrderedDict((name, OrderedDict((name, 0) for name in names)) for name in names)

# Find the co-occurrences:
    for l in document:
        for i in range(len(l)):
            for item in l[:i] + l[i + 1:]:
                occurrences[l[i]][item] += 1
                
                
    rows = ['deg','freq','deg/freq']
    word_scores = OrderedDict((name, OrderedDict((r, 0) for r in rows)) for name in names)
    
    
    for key in word_scores.keys():
        word_scores[key]['deg'] = sum(occurrences[key].values())
        word_scores[key]['freq'] = len([x for x in occurrences[key].values() if x!=0])
        if word_scores[key]['freq']!=0:
            word_scores[key]['deg/freq'] = word_scores[key]['deg']/word_scores[key]['freq']
        else:
            word_scores[key]['deg/freq']= 0
            
            
    scoredCandidates = []
    for sent in range(len(text)):
        sumis=0
        for word in word_tokenize(text[sent]):
            sumis += word_scores[word]['deg/freq']
        scoredCandidates.append((text[sent],sumis/len(word_tokenize(text[sent]))))
        
    return sorted(list(set(scoredCandidates)), key = lambda x : x[1],reverse=True)


In [450]:
df['scored'] = df['kW-candidates'].map(find_scores)

In [451]:
df['scored'][0]

[('करोड रुपैयाँ वृद्धि', 1.5888888888888888),
 ('प्रतिशत वृद्धि', 1.5833333333333335),
 ('करोड रुपैयाँ', 1.5499999999999998),
 ('करोड रुपैयाँ बढि तलब भत्ता', 1.5329870129870131),
 ('तलब भत्ता', 1.4642857142857144),
 ('वर्ष सन्', 1.4333333333333333),
 ('कर्मचारी तलब', 1.4),
 ('वर्ष अढाइ अर्ब रुपैयाँ बढि तलब भत्ता', 1.387847866419295),
 ('बढि तलब भत्ता बुझ्ने कर्मचारी', 1.372987012987013),
 ('अर्ब रुपैयाँ बढि वाषिर्क तलबभत्ता', 1.3172727272727274),
 ('प्रतिशत बढ्यो', 1.25),
 ('कुक तलब भत्तामा', 1.2254901960784315),
 (' अघिल्लो वर्ष तुलनामा सन्', 1.2166666666666668),
 ('यतिधेरै तलब भत्ता थाप्न कुक एप्पल एकमात्र कर्मचारी होलान्',
  1.1672268907563028),
 ('प्रतिशत बढे बताइए', 1.1666666666666667),
 ('प्रमुख र्यरी अधिकृत सिइओ पाउन तलब भत्ता चानचुने', 1.1369047619047619),
 ('वर्ष एप्पलमा', 1.1),
 (' कुक आधारभुत बेतनमा', 1.0588235294117647),
 ('एप्पल उच्च पदस्थ र्यरी कुक', 1.0552941176470587),
 ('एप्पल वर्तमान सिइओ टिम कुक', 1.0552941176470587),
 ('वर्षमा एप्पल', 1.05),
 (' कम्पनी प्रमुख वित्ति

In [452]:
nltk.edit_distance('गत वर्ष',' गत वर्ष सन्')/len(' गत वर्ष सन्')

0.4166666666666667

In [453]:
df['scored'][0]

[('करोड रुपैयाँ वृद्धि', 1.5888888888888888),
 ('प्रतिशत वृद्धि', 1.5833333333333335),
 ('करोड रुपैयाँ', 1.5499999999999998),
 ('करोड रुपैयाँ बढि तलब भत्ता', 1.5329870129870131),
 ('तलब भत्ता', 1.4642857142857144),
 ('वर्ष सन्', 1.4333333333333333),
 ('कर्मचारी तलब', 1.4),
 ('वर्ष अढाइ अर्ब रुपैयाँ बढि तलब भत्ता', 1.387847866419295),
 ('बढि तलब भत्ता बुझ्ने कर्मचारी', 1.372987012987013),
 ('अर्ब रुपैयाँ बढि वाषिर्क तलबभत्ता', 1.3172727272727274),
 ('प्रतिशत बढ्यो', 1.25),
 ('कुक तलब भत्तामा', 1.2254901960784315),
 (' अघिल्लो वर्ष तुलनामा सन्', 1.2166666666666668),
 ('यतिधेरै तलब भत्ता थाप्न कुक एप्पल एकमात्र कर्मचारी होलान्',
  1.1672268907563028),
 ('प्रतिशत बढे बताइए', 1.1666666666666667),
 ('प्रमुख र्यरी अधिकृत सिइओ पाउन तलब भत्ता चानचुने', 1.1369047619047619),
 ('वर्ष एप्पलमा', 1.1),
 (' कुक आधारभुत बेतनमा', 1.0588235294117647),
 ('एप्पल उच्च पदस्थ र्यरी कुक', 1.0552941176470587),
 ('एप्पल वर्तमान सिइओ टिम कुक', 1.0552941176470587),
 ('वर्षमा एप्पल', 1.05),
 (' कम्पनी प्रमुख वित्ति

In [454]:
def update(text):
    keyword = []
    count = 0
    for i in text:
        if count ==0:
            keyword.append(i[0])
        
        else:
            maxlen = max(len(i[0]),len(keyword[-1]))
            if (nltk.edit_distance(i[0],keyword[-1])/maxlen)<=0.6:
                pass
            else:
            
                keyword.append(i[0])
            
        count+=1
        
    return keyword
        

In [455]:
maxlen = max(len('करोड रुपैयाँ बढि तलब भत्ता'),len('करोड रुपैयाँ'))
nltk.edit_distance('करोड रुपैयाँ बढि तलब भत्ता','करोड रुपैयाँ')/maxlen
            

0.5384615384615384

In [456]:
df['n-scored'] = df['scored'].map(update)

In [457]:
keywords = lambda text : ' '.join(set([x for a in text[:5] for x in a.split(' ')]))

In [458]:
df['keywords'] = df['n-scored'].map(keywords)

# Similarity Between Top Sentences and Keywords

In [459]:
from difflib import SequenceMatcher

In [ ]:
गएर हेर्न सक्नेछन् पर्छ कहिल्यै आगामी ठाउने गर अहिल राख्न नभुल्नुहोस् कतिपय गरिरहनुभए दिनुभए पूरै भइरहे

In [469]:
for i in range(len(df)):
    ratio=[]
    text1 = df['keywords'][i]
    for j in range(3):
        text = summaries['Summary'+str(j+1)][i][1]
        ratio.append(SequenceMatcher(None, text, text1).ratio())
        
    index = ratio.index(max(ratio))
    
    summary_sentence = summaries['Summary'+str(index+1)][i]
    print(summary_sentence)
    
    #print(index,summary_sentence)

(0.11229021805938164, ' उन गत वर्ष उन २ अर्ब ६० करोड रुपैयाँ  बढि तलब भत्ता बुझेकी छिन् ')
(0.11229021805938164, ' उन गत वर्ष उन २ अर्ब ६० करोड रुपैयाँ  बढि तलब भत्ता बुझेकी छिन् ')
(0.12142679936370102, ' अमेरिकी कम्पनी वेपरकेड यस्तो स्मार्टफोन ल्याए छ जस तपाइँ फोन गर्न मात्र होइन की चुरोट पिउन पनि सक्नुहुन्छ ')
(0.11229021805938164, ' उन गत वर्ष उन २ अर्ब ६० करोड रुपैयाँ  बढि तलब भत्ता बुझेकी छिन् ')
(0.12142679936370102, ' अमेरिकी कम्पनी वेपरकेड यस्तो स्मार्टफोन ल्याए छ जस तपाइँ फोन गर्न मात्र होइन की चुरोट पिउन पनि सक्नुहुन्छ ')
(0.10574227052068469, 'अमेरिकी इन्टरनेट भिडियो सर्भिसनेटफि्लक्स आफ्नो सेवा विश्वब्यापी बनाए छ ')
(0.11229021805938164, ' उन गत वर्ष उन २ अर्ब ६० करोड रुपैयाँ  बढि तलब भत्ता बुझेकी छिन् ')
(0.12142679936370102, ' अमेरिकी कम्पनी वेपरकेड यस्तो स्मार्टफोन ल्याए छ जस तपाइँ फोन गर्न मात्र होइन की चुरोट पिउन पनि सक्नुहुन्छ ')
(0.10574227052068469, 'अमेरिकी इन्टरनेट भिडियो सर्भिसनेटफि्लक्स आफ्नो सेवा विश्वब्यापी बनाए छ ')
(0.07072212394277633, 'अनलाइन सेक्योरिटी कम

(0.13770128075996418, ' सन् २०१५ सेल्फी लिन क्रममा मृत्यु हुने भारतीय ही गुडिरहे रेल अगाडि रेल ट्याकमा उभिएर सेल्फी लिन क्रममा किचिए छन् त ही पिकनिक क्रममा डुंगामा उभिएर सेल्फी लिँदा पानी खसेर डुबे छन् ')
(0.2349244072740132, ' आफू न्यू वल्र्ड हृयाकिङ बताउने समूह बिहीबार बीबीसी वेबसाइटमा भए साइबर आक्रमण जिम्मा लिए छ ')
(0.07973870886491818, ' ‘ क्यानरी ’ नामक यही सुरक्षा क्यामेरा यतिबेला प्रतिस्पर्धी गुगल ड्रपक्याम नराम्ररी पछि पारेर सवै वाह्वाही बटु छ ')
(0.08922927464326982, ' नागरिक सुरक्षा र अनलाइन गोपनियताबीच कसरी सन्तुलन यम गर्न सकिन्छ भन्न बारेमा सम्बोधन गर्ने क्रममा उन यस्तो बताए हुन् ')
(0.08242272518470352, ' एक चिनियाँ कम्पनी यस्तो ड्रोन आविस्र गर छ जसभित्र एकजना मानिस आराम बसेर उड्न सक्छन् ')
(0.15069279014511616, ' अस्टि्रया साल्जबर क्षेत्रमा हाल गरिए उत्खनन क्रममा यो भेटिए हो ')
(0.11299645494189699, 'फेसबुक संस्थापक तथा सिइओ मार्क जुकरबर्ग यो वर्ष आइरनम्यान बनाएर आफ्नो सहयोगीमा नियुक्त गर्ने भए छन् ')
(0.12405516142804889, ' लक एकन लल : लक योर स्मार्टफोन एण्ड लफ लाउड आउट

# Find Accuracy with BLEU Measure